In [1]:
pip install xlrd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install cyvcf2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 4.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

from cyvcf2 import VCF

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import warnings

warnings.filterwarnings('ignore')

In [4]:
#  погода
file_path_kursk = '/kaggle/input/cvcvcv/34009.01.01.2019.31.12.2023.1.0.0.ru.utf8.00000000.xls'  # Замените на путь к вашему файлу
file_path_vorone = '/kaggle/input/cvcvcv/34123.01.01.2015.31.12.2017.1.0.0.ru.utf8.00000000.xls'
kursk = pd.read_excel(file_path_kursk, parse_dates=['Местное время'])
vorone = pd.read_excel(file_path_vorone, parse_dates=['Местное время'])

In [5]:
def prep(data):
    #изначально данные о погоде даны на каждые 30 минут, сгруппируем по дням и посчитаем статистики
    
    numeric_columns = ['T', 'P', 'U', 'Ff', 'RRR', 'sss']
    for col in numeric_columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    
    data['date'] = data['Местное время'].dt.date
    
    data['month'] = data['Местное время'].dt.month
    # Фильтруем данные по месяцам (май - сентябрь)
    filtered_data = data[(data['month'] >= 5) & (data['month'] <= 9)]
    
    agg_funcs = {
        'T': [ 'mean', 'max', 'min', 'std'], #температура
        'P': ['mean', 'max', 'min', 'std'], #давление
        'U': [ 'mean', 'max', 'min', 'std'], #влажность
        'Ff': ['mean', 'max', 'min', 'std'], #скорость ветра
        'RRR': ['sum', 'mean', 'max', 'min', 'std']    #кол-во осадков 
    }
    
    existing_vars = {col: col for col in agg_funcs.keys() if col in filtered_data.columns}
    
    daily_stats = filtered_data.groupby('date').agg(agg_funcs).reset_index()
    
    daily_stats.columns = ['date'] + [f"{var}_{stat}" for var in existing_vars.keys() for stat in agg_funcs[var]]
    daily_stats.date = pd.to_datetime(daily_stats.date)
    daily_stats['year'] = daily_stats.date.dt.year
    daily_stats['month'] = daily_stats.date.dt.month

    return daily_stats


def feat(data):
    #получение признаков погоды
    features = {
        'T_mean': data['T_mean'].mean(),
        'T_max_mean': data['T_max'].mean(),
        'T_min_mean': data['T_min'].mean(),
        'T_diff_min_max_mean': (data['T_max'] - data['T_min']).mean(),
        'T_std_mean': data['T_std'].mean(),
        'T_mean_std': data['T_mean'].std(),

        'P_mean': data['P_mean'].mean(),
        'P_std_mean': data['P_std'].mean(),
        
        'U_mean': data['U_mean'].mean(),
        'U_std_mean': data['U_std'].mean(),
        'U_min': data['U_mean'].min(),

        'Ff_mean': data['Ff_mean'].mean(),
        
        'RRR_mean': data['RRR_sum'].mean(),
        'RRR_std': data['RRR_sum'].std(),
        'RRR_max': data['RRR_sum'].max(),
    }
    
    # Выявление аномалий
    features.update({
        'days_high_temp': (data['T_max'] > 30).sum(),  # Кол-во дней с температурой > 30°C
        'days_high_precip': (data['RRR_sum'] > 20).sum(),  # Кол-во дней с осадками > 20 мм
        'days_high_wind': (data['Ff_mean'] > 15).sum(),  # Кол-во дней с ветром > 15 м/с
        'days_high_humidity': (data['U_mean'] < 40).sum(),  # Кол-во дней с влажность < 40

        
        'anomalous_days_fraction': ((data['T_max'] > 30) | 
                                    (data['RRR_sum'] > 20) | 
                                    (data['Ff_mean'] > 15) |
                                   (data['U_mean'] < 40)).mean(),  # Доля дней с аномалиями
    })
    
    features.update({
        'thermal_sum_above_10': (data['T_mean'][data['T_mean'] > 10]).sum(),  #  активные температуры
        'dry_days': (data['RRR_sum'] == 0).sum(),  # Количество сухих дней (без осадков)
        
        # Периоды засухи (максимальное число последовательных дней без осадков)
        'max_dry_spell': (data['RRR_sum'] == 0).astype(int).groupby((data['RRR_sum'] > 0).cumsum()).cumsum().max(),

        #  (максимальное число последовательных дней с осадками)
        'max_rainy_spell': (data['RRR_sum'] > 0).astype(int).groupby((data['RRR_sum'] == 0).cumsum()).cumsum().max(),

        
    })
    
    df = pd.DataFrame([features])
    return df

kursk = prep(kursk)
vorone = prep(vorone)
weather = pd.concat([vorone, kursk]).reset_index(drop=True)


def calculate_features(data):

    #итоговые признаки погоды (сгруппированные по месяцам, коэф селянинова)
    
    result = {}
    months_map = {5: 'may', 6: 'june', 7: 'july', 8: 'august', 9: 'september'}
    
    # Группируем данные по годам
    for year, group in data.groupby('year'):
        year_features = {}  # Признаки для одного года
        
        # Обрабатываем данные для каждого месяца
        for month, month_group in group.groupby('month'):
            if month in months_map:  # Проверяем, если месяц в нужном диапазоне
                month_features = feat(month_group).iloc[0].to_dict()  # Получаем признаки
                # Добавляем суффиксы к названиям признаков
                month_features = {f"{key}_{months_map[month]}": value for key, value in month_features.items()}
                year_features.update(month_features)
        
        #
        temp_above_10 = group[group['T_mean'] > 10]
        R = temp_above_10['RRR_sum'].sum()  # Сумма осадков
        sum_t = temp_above_10['T_mean'].sum()  # Сумма температур
        if sum_t > 0:  
            K = R * 10 / sum_t
        else:
            K = 0  # Если нет температур >10°C
        year_features['K_total'] = K  # Коэффициент Селянинова
        
        year_features['year'] = year
        result[year] = year_features
    
    return pd.DataFrame.from_dict(result, orient='index').reset_index(drop=True)

monthly_features = calculate_features(weather)

In [6]:
monthly_features

,T_mean_may,T_max_mean_may,T_min_mean_may,T_diff_min_max_mean_may,T_std_mean_may,T_mean_std_may,P_mean_may,P_std_mean_may,U_mean_may,U_std_mean_may,...,days_high_precip_september,days_high_wind_september,days_high_humidity_september,anomalous_days_fraction_september,thermal_sum_above_10_september,dry_days_september,max_dry_spell_september,max_rainy_spell_september,K_total,year
0,16.394355,21.619355,10.961290,10.658065,4.102964,4.146812,758.747581,0.947608,56.665323,16.123955,...,0.0,0.0,5.0,0.200000,527.407143,27.0,24.0,2.0,0.664670,2015
1,15.191475,19.648387,10.922581,8.725806,3.375741,3.325459,759.195161,0.939171,73.285714,14.478881,...,0.0,0.0,0.0,0.000000,309.150000,22.0,14.0,3.0,0.860412,2016
2,13.960484,19.016129,8.851613,10.164516,3.917167,4.599116,760.422581,1.215354,57.241935,15.607074,...,1.0,0.0,0.0,0.066667,409.737500,24.0,15.0,2.0,0.870396,2017
3,16.771774,20.822581,12.645161,8.177419,3.173412,3.926088,760.150806,1.082444,68.427419,13.688871,...,0.0,0.0,3.0,0.100000,355.162500,19.0,16.0,6.0,1.421015,2019
4,11.645968,15.258065,8.341935,6.916129,2.621112,3.153255,759.314919,1.096419,68.967742,14.038425,...,0.0,0.0,9.0,0.366667,485.200000,26.0,9.0,2.0,2.022907,2020
5,14.707258,18.832258,10.522581,8.309677,3.193212,4.418282,759.391935,1.296648,63.431452,14.281650,...,2.0,0.0,0.0,0.066667,216.650000,15.0,13.0,13.0,2.036558,2021
6,12.132258,16.151613,7.825806,8.325806,3.257029,2.558562,760.731048,1.350447,55.129032,13.938660,...,4.0,0.0,0.0,0.133333,230.500000,10.0,4.0,9.0,1.944647,2022
7,14.276210,18.470968,9.758065,8.712903,3.373228,4.279168,765.373790,0.854064,54.274194,12.523520,...,0.0,0.0,0.0,0.000000,494.187500,27.0,21.0,2.0,2.562050,2023


In [7]:
def pca(x, n_components, weather='weather'):
    x = x.fillna(0)
    original_index = x.index

    if weather=='weather':
        x = x.drop(columns='year')
        
    scaler = StandardScaler()
    x = scaler.fit_transform(x)
    pca = PCA(n_components=n_components)
    x = pca.fit_transform(x)
    x = pd.DataFrame(x, index=original_index, columns=[f'{weather}{i+1}' for i in range(n_components)])
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_variance = explained_variance_ratio.cumsum()
    print(explained_variance_ratio, cumulative_variance)
    if weather=='weather':
        x['year'] = monthly_features['year'].values
    
    return x

## Генотипы

In [9]:
vcf_file = "/kaggle/input/cvcvcv/genotypes.vcf"
vcf = VCF(vcf_file, strict_gt=True)  #

samples = vcf.samples  
genotypes = []
filtered_genotypes = []
filtered_variants = []

for variant in vcf:
    if variant.QUAL > 900:
        gt = variant.genotypes
        numeric_gt = [sum(alleles[:2]) if alleles[0] != -1 else None for alleles in gt]
        filtered_genotypes.append(numeric_gt)
        filtered_variants.append(f"{variant.CHROM}_{variant.POS}")

genotypes_df = pd.DataFrame(filtered_genotypes, columns=samples, index=filtered_variants).T.fillna(0)
genotypes_df.shape

(99, 35693)

## Фенотипы

In [10]:
phenotype_df = pd.read_csv('/kaggle/input/cvcvcv/phenotypes.tsv', sep='\t').reset_index(drop=True)
phenotype_long = phenotype_df.melt(id_vars=["sample"], var_name="year", value_name="yield")
phenotype_long.dropna(subset=["yield"], inplace=True)

phenotype_long['year'] = phenotype_long['year'].astype(int)
phenotype_long.head(3)

,sample,year,yield
14,PS000076,2015,114.0
15,PS000033,2015,116.0
16,PS000048,2015,99.0


#### PCA

In [11]:
def pca(x, n_components, weather='weather'):
    x = x.fillna(0)
    original_index = x.index

    if weather=='weather':
        x = x.drop(columns='year')
        
    scaler = StandardScaler()
    x = scaler.fit_transform(x)
    pca = PCA(n_components=n_components)
    x = pca.fit_transform(x)
    x = pd.DataFrame(x, index=original_index, columns=[f'{weather}{i+1}' for i in range(n_components)])
    explained_variance_ratio = pca.explained_variance_ratio_.sum()
    # cumulative_variance = explained_variance_ratio.cumsum()
    print(explained_variance_ratio)
    if weather=='weather':
        x['year'] = monthly_features['year'].values
    
    return x

In [12]:
monthly_features_pca = pca(monthly_features,8)
genotypes_df_pca = pca(genotypes_df,90, 'genome')


1.0
0.9766425213643786


### Если просто предсказывать средним значением

In [13]:
# Отделяем 2023 год (тестовая выборка)
test_data = phenotype_long[phenotype_long['year'] > 2022]

# Остальные годы (тренировочная выборка)
train_data = phenotype_long[phenotype_long['year'] <= 2022]

# Рассчитываем среднее значение  по тренировочной выборке
mean_yield = train_data['yield'].mean()

y_pred = [mean_yield] * len(test_data)

y_true = test_data['yield'].values

# Рассчитываем rMSE
mse = mean_squared_error(y_true, y_pred,squared= False)

print(f"Среднее значение  урожайности: {mean_yield}")
print(f"rMSE для тестовой выборки (2023 год): {mse}")


Среднее значение  урожайности: 104.09883720930233
rMSE для тестовой выборки (2023 год): 14.237427521759454


## Соеденим и получим итоговые признаки

In [14]:
vegetation = pd.read_csv('/kaggle/input/cvcvcv/vegetation.tsv', sep='\t').reset_index(drop=True)
vegetation.head(1)

,sample,vegetation
0,PS000196,100


In [15]:
X_ = genotypes_df_pca.reset_index(names='sample').merge(phenotype_long, on='sample').merge(vegetation, on='sample').merge(monthly_features_pca, on='year')
print(X_.shape)

(359, 102)


In [16]:
X_.head()

,sample,genome1,genome2,genome3,genome4,genome5,genome6,genome7,genome8,genome9,...,yield,vegetation,weather1,weather2,weather3,weather4,weather5,weather6,weather7,weather8
0,PS000026,-19.685537,-0.767683,44.646656,24.616256,-19.185397,13.63206,-30.990835,9.190035,-12.461585,...,140.0,110,-7.948856,-4.734405,-2.459280,3.179346,-0.847350,4.119559,-2.547785,1.113817e-15
1,PS000026,-19.685537,-0.767683,44.646656,24.616256,-19.185397,13.63206,-30.990835,9.190035,-12.461585,...,124.0,110,-1.123685,-4.658325,-4.385831,-0.106670,-3.982249,-3.181954,4.082574,1.113817e-15
2,PS000026,-19.685537,-0.767683,44.646656,24.616256,-19.185397,13.63206,-30.990835,9.190035,-12.461585,...,115.0,110,-2.177201,1.822792,-0.697921,2.901639,7.927438,-0.876646,2.279972,1.113817e-15
3,PS000026,-19.685537,-0.767683,44.646656,24.616256,-19.185397,13.63206,-30.990835,9.190035,-12.461585,...,121.0,110,-4.763318,5.335375,8.467748,2.178880,-3.179618,-1.844949,0.066395,1.113817e-15
4,PS000026,-19.685537,-0.767683,44.646656,24.616256,-19.185397,13.63206,-30.990835,9.190035,-12.461585,...,117.0,110,4.773086,-1.805586,4.523063,-4.373646,0.106292,5.276951,2.392950,1.113817e-15


In [17]:
# добавим произведения компонент генотипа и компонент погоды
interaction_terms = []

for genotype_col in genotypes_df_pca.columns:
    for weather_col in monthly_features_pca.columns[:-1]:
        interaction_term = X_[genotype_col] * X_[weather_col]
        interaction_terms.append(interaction_term)
len(interaction_terms)

720

In [18]:
#понизим размерность признаков взаимодействий до 40
interaction_df = pd.DataFrame(interaction_terms).T
interaction_df.columns = [f'Interaction_{genotype}_{weather}' for genotype in genotypes_df_pca.columns for weather in monthly_features_pca.columns[:-1]]
interaction_df = pca(interaction_df,40, 'comb')
X_ = pd.concat([X_, interaction_df], axis=1)
X_

0.1499078951550382


,sample,genome1,genome2,genome3,genome4,genome5,genome6,genome7,genome8,genome9,...,comb31,comb32,comb33,comb34,comb35,comb36,comb37,comb38,comb39,comb40
0,PS000026,-19.685537,-0.767683,44.646656,24.616256,-19.185397,13.632060,-30.990835,9.190035,-12.461585,...,-1.682253,-0.072185,0.859081,2.090935,-2.546866,0.238681,1.175695,-2.969436,-0.018694,-1.112051
1,PS000026,-19.685537,-0.767683,44.646656,24.616256,-19.185397,13.632060,-30.990835,9.190035,-12.461585,...,2.266310,-1.043840,-0.858310,-0.712681,-1.895013,-0.079112,0.988483,-1.108806,0.598603,-1.012754
2,PS000026,-19.685537,-0.767683,44.646656,24.616256,-19.185397,13.632060,-30.990835,9.190035,-12.461585,...,-0.286199,0.386382,-0.250859,0.817404,-0.517335,-1.258012,-0.145932,-1.374267,1.471834,0.894271
3,PS000026,-19.685537,-0.767683,44.646656,24.616256,-19.185397,13.632060,-30.990835,9.190035,-12.461585,...,-1.915652,1.538308,0.182473,2.537089,0.272795,-1.890933,-2.347843,-0.877727,1.258191,2.461481
4,PS000026,-19.685537,-0.767683,44.646656,24.616256,-19.185397,13.632060,-30.990835,9.190035,-12.461585,...,2.942393,-1.630449,-2.043622,0.115889,1.433088,0.482046,-0.666525,2.165646,-0.831184,0.028406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,PS000568,-45.382027,-33.896695,-7.292698,-56.122963,-11.323841,-44.769365,19.495406,43.878492,99.007411,...,-0.124216,-5.594826,-0.539277,-3.469393,2.060101,0.304003,-1.655206,3.724341,-1.581313,0.919575
355,PS000568,-45.382027,-33.896695,-7.292698,-56.122963,-11.323841,-44.769365,19.495406,43.878492,99.007411,...,2.678897,-1.435552,0.030045,1.586827,2.282364,-4.221533,0.095150,-3.597924,-0.541740,1.108015
356,PS000570,-36.480579,-47.207258,-15.918328,36.542710,-94.288203,7.750778,89.656312,-32.905891,93.175745,...,3.788498,1.073089,0.242789,-4.228583,-0.728803,2.353657,-0.024274,-0.746520,-2.355462,1.303660
357,PS000570,-36.480579,-47.207258,-15.918328,36.542710,-94.288203,7.750778,89.656312,-32.905891,93.175745,...,2.406334,-0.499515,-0.649112,-0.368661,2.904480,1.166983,-2.413163,1.414010,-0.466100,1.154622


In [19]:
train_data = X_[X_['year'] < 2022]  # Все года до 2022
test_data = X_[X_['year'] >= 2022]   # 2022-2023

# Разделяем признаки (X) и целевую переменную (y)
X_train = train_data.drop(columns=['yield', 'year', 'sample'])  # 
y_train = train_data['yield']                # 

X_test = test_data.drop(columns=['yield', 'year', 'sample'])   # 
y_test = test_data['yield']                  # 

print(f"Размер train: {X_train.shape}, {y_train.shape}")
print(f"Размер test: {X_test.shape}, {y_test.shape}")

Размер train: (325, 139), (325,)
Размер test: (34, 139), (34,)


In [30]:
#отберем признаки с помощью L1
lasso = Lasso(alpha=3)  #
lasso.fit(X_train, y_train)

# print(f"Отобранные признаки: {X_train.columns[lasso.coef_ != 0]}")
print('кол-во признаков',len(lasso.coef_ != 0))
X_train = X_train[X_train.columns[lasso.coef_ != 0]]
X_test = X_test[X_test.columns[lasso.coef_ != 0]]

кол-во признаков 81


In [32]:
# model = CatBoostRegressor(verbose=1000, random_state=1, iterations=500, max_depth=2)
model = SVR(C=4)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)


print(f"RMSE: {rmse } MAE: {mae}")

RMSE: 10.95292266377959 MAE: 7.038220256518391
